In [1]:
import pandas as pd 
import cv2, numpy as np
from scipy.stats import norm
from scipy.misc import imread, imsave
import re
import idx2numpy as idx
from copy import copy
import os
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.image as mpimg
%matplotlib inline

from sklearn.metrics import precision_recall_fscore_support, f1_score, accuracy_score, precision_score, recall_score
from sklearn.manifold import TSNE

# keras
np.random.seed(13)
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Reshape, Activation, SimpleRNN, GRU, LSTM, Convolution1D, \
                         MaxPooling1D, Merge, Dropout, Input
from IPython.display import SVG
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
#from keras.utils.visualize_util import model_to_dot, plot
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.layers import Input, Flatten, Dense, Dropout, Activation, Lambda, Layer # keras.layers.core 
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.applications.vgg16 import VGG16
from keras import metrics
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# logging
import logging
from importlib import reload
reload(logging)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Using TensorFlow backend.


In [36]:
rootpath = "../EgyptianHieroglyphDataset/MyTrainTest"
original_dim = (75, 50) # without convolutional layers, use 75 * 50
batch_size = 32 * 2 # this has to be the same as the param value in the analysis code

In [3]:
tr_rootpath = os.path.join(rootpath, "train")
symbol_df = [x for x in os.listdir(tr_rootpath) if re.search(r"UNKNOWN|^\.", x) == None]
symbol_df = pd.DataFrame(sorted(symbol_df))
symbol_df.reset_index(inplace = True)
symbol_df.columns = ["symbol_num", "symbol"]
symbol_df.tail(1)
# TODO pickle symbol_df; load if when available

,symbol_num,symbol
170,170,Z7


In [14]:
x_train = np.load("/home/ubuntu/egypt/data/after_augmentation_but_wrong_shape/x_train.pkl")
y_train = np.load("/home/ubuntu/egypt/data/after_augmentation_but_wrong_shape/y_train.pkl")

### check the records
Need to get rid of these three images so that total training data is divisible by batch size  
These correspond to: gen3_410397_O50.png, gen3_410385_O50.png, gen3_410365_O50.png

In [26]:
pd.Series(np.argmax(y_train[41814:41817, :], 1), name = "temp") 

0    113
1    113
2    113
Name: temp, dtype: int64

### fix x_train

In [29]:
# testing code to see if things work
# temp = np.array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7])
# temp = np.reshape(temp, (7, 1, 2, 2))
# np.vstack([temp[:3], temp[5:]])

In [34]:
print(x_train.shape)
x_train = np.vstack([x_train[:41814], x_train[41817:]])
print(x_train.shape, x_train.shape[0] % batch_size)

(55875, 1, 75, 50)
(55872, 1, 75, 50)


In [38]:
x_train.dump("/home/ubuntu/egypt/data/x_train.pkl")

### fix y_train

In [40]:
print(y_train.shape)
y_train = np.vstack([y_train[:41814], y_train[41817:]])
print(y_train.shape, y_train.shape[0] % batch_size)

(55875, 171)
(55872, 171) 0


In [41]:
y_train.dump("/home/ubuntu/egypt/data/y_train.pkl")